In [1]:
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-azure-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-azure-openai

In [2]:
#!pip uninstall llama_index==0.8.9
!pip install llama-index

In [3]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [4]:
# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = "sk----"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output


# define LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [5]:

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


#azure_endpoint = "https://.openai.azure.com/"
#api_version = "2023-07-01-preview"


api_key = "e------------------------------"
azure_endpoint = "https://azure----------------------.openai.azure.com/"

#"text-embedding-ada-002"

#"gpt-35-turbo"

lc_llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)


In [6]:
#%pip install ipython-ngql nebula3-python

In [7]:
from llama_index.core import Settings

Settings.llm = lc_llm 
Settings.embed_model = embedding_llm
Settings.chunk_size = 512

In [8]:
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ['GRAPHD_HOST'] = "127.0.0.1"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "rag_workshop"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [9]:
%load_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,rag_workshop


In [10]:
%ngql CREATE SPACE IF NOT EXISTS rag_workshop(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [11]:
%%ngql
USE rag_workshop;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [12]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [13]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [14]:
from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Tesla Cybertruck"], auto_suggest=False
)

In [15]:
from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings

In [16]:
kg_index.storage_context.persist(persist_dir='./storage_graph')

In [17]:
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex,  VectorStoreIndex
from llama_index.core import ServiceContext, set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=lc_llm,
    embed_model=embedding_llm,
)

set_global_service_context(service_context)

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

INFO:llama_index.core.indices.loading:Loading all indices.


/tmp/ipykernel_141056/577445147.py:6: DeprecationWarning: Call to deprecated function (or staticmethod) from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [18]:
#!pip install pydantic==1.10.13
#!pip install docarray==0.32.1
#!pip install pydantic==1.10.13
%ngql SHOW HOSTS

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Host,Port,Status,Leader count,Leader distribution,Partition distribution,Version
0,storaged0,9779,ONLINE,0,No valid partition,No valid partition,3.6.0
1,storaged1,9779,ONLINE,0,No valid partition,No valid partition,3.6.0
2,storaged2,9779,ONLINE,1,rag_workshop:1,rag_workshop:1,3.6.0


In [19]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=lc_llm,
)

In [20]:
#re=nl2kg_query_engine.query("SHOW HOSTS")

In [21]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [22]:
response_graph_rag = kg_index_query_engine.query("What is Cybertruck")

display(f"<b>{response_graph_rag}</b>")

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Extracted keywords: ['Cybertruck']
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 2e195124-02e2-4a24-aa06-f4f29d7eb6c1: ==== Range extender ====
The dual-motor and tri-motor configurations can be o...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 0e0ffd60-0029-4fe5-b1af-1464a468929c: The Tesla Cybertruck is a battery electric medium duty full-size pickup truck...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: e6163b2d-de85-466e-93fa-0c3a45a270fd: === Pilot production ===

Franz von Holzhausen drove a prototype to the Peter...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Cybertruck{name: Cyber

'<b>Cybertruck is a medium duty full-size pickup truck built by Tesla, Inc. since 2023. It has a triangular body with flat sheet stainless steel panels. It offers up to 9.6 kW of vehicle-to-load (V2L) continuous AC power through five outlets, supports up to 11.5 kW AC output for vehicle-to-home (V2H), or vehicle-to-vehicle (V2V) charging, and uses a 48-volt electrical system. The Cybertruck has three models: the tri-motor all-wheel drive (AWD) "Cyberbeast", a dual-motor AWD model, and a rear-wheel drive (RWD) model, with EPA range estimates of 250–340 miles (400–550 km), varying by model. It also has features such as wade mode, range extender, and armor glass windows.</b>'

In [28]:
response_graph_rag = kg_index_query_engine.query("Who is Musk?")

display(f"<b>{response_graph_rag}</b>")

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Extracted keywords: ['Musk']
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: d5cf6b11-3513-4ee0-8eb1-ddac820f0501: === Reservations ===

Beginning in November 2019, Tesla accepted Cybertruck r...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: de08cfb8-806f-4c92-8642-bd8fc7528a01: == Design ==


=== Inspiration and styling ===

According to Musk, the design...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Musk{name: Musk} -[relationship:{relationship: Inspired by}]-> Wet nellie{name: Wet nellie} -[relationship:{relationship: Driven by}]-> James bond{name: James bond}
Musk{name: Musk} -[relationship:{relationship: Inspired by}]

"<b>Musk is a person mentioned in the context information who is involved in the design process of the Cybertruck, stated that final specifications and pricing would be materially different from those unveiled on the concept vehicle in 2019, and was inspired by Blade Runner and Wet Nellie for the design of the Cybertruck. Musk also used his son's quote as an inspiration for the design and wanted the Cybertruck to have all the utility of a pick-up truck.</b>"

In [30]:
response_graph_rag = kg_index_query_engine.query("Is Cybertruck expensive?")

display(f"<b>{response_graph_rag}</b>")

INFO:httpx:HTTP Request: POST https://azure-openai-98325.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Extracted keywords: ['expensive', 'Cybertruck']
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 2e195124-02e2-4a24-aa06-f4f29d7eb6c1: ==== Range extender ====
The dual-motor and tri-motor configurations can be o...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 0e0ffd60-0029-4fe5-b1af-1464a468929c: The Tesla Cybertruck is a battery electric medium duty full-size pickup truck...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: e6163b2d-de85-466e-93fa-0c3a45a270fd: === Pilot production ===

Franz von Holzhausen drove a prototype to the Peter...
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
Cybertruc

'<b>The pricing for the Cybertruck varies depending on the model. The base price for the single-motor RWD model is $60,990 and is set to be available in 2025. The all-wheel drive model is set to be available in 2024 at a starting price of $79,990, while the tri-motor Cyberbeast variant is offered at $99,990, also set to be available in 2024. Compared to the 2019 unveiling of the concept Cybertruck, base prices had risen by $21,000–39,000, depending on the model, an increase of 53% to 64%. Therefore, it can be said that the Cybertruck is relatively expensive, but the pricing varies depending on the model.</b>'